In [1]:
from ReadData import ReadData
from Build_Model import BuildModelClassify
import tensorflow as tf
from PIL import Image
import os
from IPython.display import display
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import ast

In [2]:
if tf.test.gpu_device_name():
    print('GPU available:', tf.test.gpu_device_name())
    
else:
    print("No GPU available. Please make sure you have TensorFlow configured to use GPU.")

GPU available: /device:GPU:0


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
# Đọc file CSV vào DataFrame
file_path = 'D:/Cuong/private/work/Python/Project/tensorflow/HackAthon/output.csv'

# Chunksize - số lượng dòng đọc mỗi lần
chunksize = 300

# Khởi tạo DataFrame trống cho kết quả
result_data = pd.DataFrame()
count = 0
# Đọc file CSV theo từng phần nhỏ (chunk)
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    if not count == 0:
        # Lọc ra 500 hàng có giá trị masked = 0 và 500 hàng có giá trị masked = 1
        filtered_data_1 = chunk[chunk['emotion'] == 'Happiness'].head(215 - len(result_data[result_data['emotion'] == 'Happiness']))
        filtered_data_2 = chunk[chunk['emotion'] == 'Neutral'].head(215 - len(result_data[result_data['emotion'] == 'Neutral']))
        filtered_data_3 = chunk[chunk['emotion'] == 'Sadness'].head(215 - len(result_data[result_data['emotion'] == 'Sadness']))
        filtered_data_4 = chunk[chunk['emotion'] == 'Anger'].head(215 - len(result_data[result_data['emotion'] == 'Anger']))
        filtered_data_5 = chunk[chunk['emotion'] == 'Surprise'].head(215 - len(result_data[result_data['emotion'] == 'Surprise']))
        filtered_data_6 = chunk[chunk['emotion'] == 'Disgust'].head(215 - len(result_data[result_data['emotion'] == 'Disgust']))
        filtered_data_7 = chunk[chunk['emotion'] == 'Fear'].head(215 - len(result_data[result_data['emotion'] == 'Fear']))
    else:
        filtered_data_1 = chunk[chunk['emotion'] == 'Happiness'].head(215)
        filtered_data_2 = chunk[chunk['emotion'] == 'Neutral'].head(215)
        filtered_data_3 = chunk[chunk['emotion'] == 'Sadness'].head(215)
        filtered_data_4 = chunk[chunk['emotion'] == 'Anger'].head(215)
        filtered_data_5 = chunk[chunk['emotion'] == 'Surprise'].head(215)
        filtered_data_6 = chunk[chunk['emotion'] == 'Disgust'].head(215)
        filtered_data_7 = chunk[chunk['emotion'] == 'Fear'].head(215)
    # Kết hợp lại thành DataFrame mới
    result_data = pd.concat([result_data, filtered_data_1,filtered_data_2, filtered_data_3, filtered_data_4, filtered_data_5, filtered_data_6, filtered_data_7])
    count = 1

    # Kiểm tra xem đã đạt được 500 hàng cho mỗi loại masked chưa
    if (len(result_data[result_data['emotion'] == 'Happiness']) >= 215 and
            len(result_data[result_data['emotion'] == 'Neutral']) >= 215 and 
            len(result_data[result_data['emotion'] == 'Sadness']) >= 215 and 
            len(result_data[result_data['emotion'] == 'Anger']) >= 215 and 
            len(result_data[result_data['emotion'] == 'Surprise']) >= 215 and 
            len(result_data[result_data['emotion'] == 'Disgust']) >= 215 and 
            len(result_data[result_data['emotion'] == 'Fear']) >= 215):
        break

In [5]:
chunk = 0
filtered_data_1 = 0
filtered_data_2 = 0
filtered_data_3 = 0
filtered_data_4 = 0
filtered_data_5 = 0
filtered_data_6 = 0
filtered_data_7 = 0

In [6]:
result_data['emotion'].value_counts()

emotion
Happiness    215
Neutral      215
Sadness      215
Anger        215
Surprise     215
Disgust      132
Fear         114
Name: count, dtype: int64

In [7]:
def tranformX(X):
    data_arrays = []
    for value in X:
        data_array = np.array(ast.literal_eval(value))
        data_arrays.append(data_array)
    return np.array(data_arrays)

In [8]:
build_model = BuildModelClassify()
model = build_model.build()

In [9]:
def tranformDF(df):
    tasks = ['age', 'race', 'masked', 'skintone', 'emotion', 'gender']
    for task in tasks:
        if task == 'age':
            df['age'] = df['age'].astype(str).map({'20-30s': 0,
                                               '40-50s': 1,
                                               'Kid': 2,
                                               'Senior': 3,
                                               'Teenager': 4,
                                               'Baby': 5})
        elif task == 'masked':
            # Lấy nhãn 'masked' và chuyển đổi sang chuỗi
            df['masked'] = df['masked'].astype(str).map({'unmasked': 0, 'masked': 1})
    
        elif task == 'skintone':
            df['skintone'] = df['skintone'].astype(str).map({'mid-light': 0,
                                                    'light': 1,
                                                    'mid-dark': 2,
                                                    'dark': 3})
    
        elif task == 'emotion':
            df['emotion'] = df['emotion'].astype(str).map({'Happiness': 0,
                                                   'Neutral': 1,
                                                   'Sadness': 2,
                                                   'Anger': 3,
                                                   'Surprise': 4,
                                                   'Disgust': 5,
                                                   'Fear': 6})
    
        elif task == 'race':
            df['race'] = df['race'].astype(str).map({'Mongoloid': 0,
                                                'Caucasian': 1,
                                                'Negroid': 2})
    
        elif task == 'gender':
            df['gender'] = df['gender'].astype(str).map({'Female': 0,
                                                  'Male': 1})

In [10]:
tranformDF(result_data)

In [11]:
result_data

,file_name,height,width,bbox,age,race,masked,skintone,emotion,gender,var1,var2,var3,var4,array_images
2,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,2000,1333,"[419.93871061403877, 269.1250391680045, 377.19...",0,0,0,1,0,0,419.938711,269.125039,377.190690,491.115671,"[[[0.35555555555555557, 0.23555555555555555, 0..."
4,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,2000,1333,"[549.169724453414, 92.52040334013152, 306.8821...",0,1,0,0,0,0,549.169724,92.520403,306.882136,437.365818,"[[[0.06222222222222222, 0.06222222222222222, 0..."
6,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,2000,1333,"[640.2165901410499, 619.7806050101991, 359.991...",0,0,0,1,0,0,640.216590,619.780605,359.991091,440.243879,"[[[0.37777777777777777, 0.3244444444444444, 0...."
7,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,2000,1333,"[468.0460743801641, 174.78512396694236, 326.43...",1,1,0,0,0,0,468.046074,174.785124,326.432645,417.776860,"[[[0.4088888888888889, 0.2577777777777778, 0.1..."
8,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,1333,2000,"[1277.1383338280361, 140.98490082153265, 248.8...",0,0,0,1,0,0,1277.138334,140.984901,248.888240,351.588007,"[[[1.1333333333333333, 1.1333333333333333, 1.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14614,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,1024,1024,"[253.22926829268306, 161.10731707317052, 566.5...",0,1,0,1,5,0,253.229268,161.107317,566.512195,788.858537,"[[[0.68, 0.47555555555555556, 0.28444444444444..."
14640,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,1024,1024,"[187.87793427230048, 153.4366197183098, 609.74...",5,1,0,1,6,0,187.877934,153.436620,609.746479,715.774648,"[[[0.4311111111111111, 0.30666666666666664, 0...."
14803,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,1024,1024,"[178.7042253521122, 167.5727699530506, 558.286...",1,1,0,2,5,1,178.704225,167.572770,558.286385,795.483568,"[[[0.7422222222222222, 0.7244444444444444, 0.7..."
14819,D:/Cuong/HackAthon/mnt/md0/projects/sami-hacka...,1024,1024,"[173.49295774647752, 153.07981220657268, 585.7...",0,0,0,1,5,0,173.492958,153.079812,585.727700,771.793427,"[[[0.1688888888888889, 0.13777777777777778, 0...."


In [12]:
from sklearn.utils import resample
# Down sample emotion 
happiness = result_data[result_data['emotion'] == 0]
neutral = result_data[result_data['emotion'] == 1]
sadness = result_data[result_data['emotion'] == 2]
anger = result_data[result_data['emotion'] == 3]
surprise = result_data[result_data['emotion'] == 4]
disgust = result_data[result_data['emotion'] == 5]
fear = result_data[result_data['emotion'] == 6]

news_disgust = resample(disgust, replace=True, n_samples=125,random_state=10)
news_fear = resample(fear, replace=True, n_samples=125,random_state=10)
result_data = pd.concat([happiness, neutral,sadness,anger,surprise,news_disgust,news_fear])

In [13]:
happiness, neutral, sadness,anger,surprise,disgust,fear,news_disgust,news_fear = 0,0,0,0,0,0,0,0,0

In [14]:
result_data = result_data.drop(['file_name', 'height', 'width', 'bbox','var1','var2','var3','var4'], axis=1)

In [15]:
train, test = train_test_split(result_data, test_size=0.1, shuffle=True)

In [16]:
result_data = 0

In [17]:
X_train, X_test = train['array_images'], test['array_images']
# Reformat X_train to numpy array (224,224,3)
X_train = tranformX(X_train)
X_test = tranformX(X_test)

# One hot labels
label_age = to_categorical(np.array(train['age']), 6)
label_race = to_categorical(np.array(train['race']), 3)
label_masked = to_categorical(np.array(train['masked']), 2)
label_skintone = to_categorical(np.array(train['skintone']), 4)
label_emotion = to_categorical(np.array(train['emotion']), 7)
label_gender = to_categorical(np.array(train['gender']), 2)

label_age_test = to_categorical(np.array(test['age']), 6)
label_race_test = to_categorical(np.array(test['race']), 3)
label_masked_test = to_categorical(np.array(test['masked']), 2)
label_skintone_test = to_categorical(np.array(test['skintone']), 4)
label_emotion_test = to_categorical(np.array(test['emotion']), 7)
label_gender_test = to_categorical(np.array(test['gender']), 2)

model.load_weights('train1/FUN.h5')

# Only train age task
for layer in model.layers:
    if layer.name == 'Emotion1' or layer.name == 'Emotion' :
        layer.trainable = True
    else :
        layer.trainable = False


# Train 
model.fit(
    X_train,
    {
        'Age': label_age,
        'Race': label_race,
        'Masked': label_masked,
        'Skintone': label_skintone,
        'Emotion': label_emotion,
        'Gender': label_gender
    },
    epochs=30,
    validation_data=(X_test, 
                     {
                        'Age': label_age_test,
                        'Race': label_race_test,
                        'Masked': label_masked_test,
                        'Skintone': label_skintone_test,
                        'Emotion': label_emotion_test,
                        'Gender': label_gender_test
                     }),
    
)

Epoch 1/30
38/38 [==============================] - 48s 429ms/step - loss: 4.7850 - Age_loss: 0.8393 - Race_loss: 0.5505 - Masked_loss: 0.1060 - Skintone_loss: 0.7312 - Emotion_loss: 2.1171 - Gender_loss: 0.4410 - Age_accuracy: 0.7374 - Race_accuracy: 0.7617 - Masked_accuracy: 0.9773 - Skintone_accuracy: 0.6997 - Emotion_accuracy: 0.3364 - Gender_accuracy: 0.8037 - val_loss: 3.8886 - val_Age_loss: 0.8003 - val_Race_loss: 0.3737 - val_Masked_loss: 0.1350 - val_Skintone_loss: 0.5969 - val_Emotion_loss: 1.6718 - val_Gender_loss: 0.3109 - val_Age_accuracy: 0.7218 - val_Race_accuracy: 0.8421 - val_Masked_accuracy: 0.9699 - val_Skintone_accuracy: 0.7444 - val_Emotion_accuracy: 0.4135 - val_Gender_accuracy: 0.8496
Epoch 2/30
38/38 [==============================] - 10s 262ms/step - loss: 4.0118 - Age_loss: 0.7773 - Race_loss: 0.4451 - Masked_loss: 0.0718 - Skintone_loss: 0.6163 - Emotion_loss: 1.7249 - Gender_loss: 0.3763 - Age_accuracy: 0.7542 - Race_accuracy: 0.8054 - Masked_accuracy: 0.976

In [18]:
model.save_weights('train1/FUN.h5')

In [18]:
X_train, X_test = 0,0